In [ ]:
#import os
#path = '/content/drive/MyDrive/Colab Notebooks/CaImAn/demos/notebooks/'

#change current directory
#os.chdir(path)


In [20]:
from datetime import datetime
import scipy.io as sio
import re
import os
import h5py
import csv
import tensorflow as tf
import time
import logging
import zipfile
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('default')
import numpy as np
from moviepy.editor import *
import smtplib

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3801088/45929032 bytes (8.3%)7593984/45929032 bytes (16.5%)11534336/45929032 bytes (25.1%)15245312/45929032 bytes (33.2%)18817024/45929032 bytes (41.0%)22536192/45929032 bytes (49.1%)26066944/45929032 bytes (56.8%)29876224/45929032 bytes (65.0%)33619968/45929032 bytes (73.2%)37388288/45929032 bytes (81.4%)40992768/45929032 bytes (89.3%)44843008/45929032 bytes (97.6%)

# 初期設定

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks'

# Install CaImAn
#!git clone https://github.com/flatironinstitute/CaImAn.git
!git clone https://github.com/etterguillaume/CaImAn.git
#%cd '/content/CaImAn/'
%cd '/content/drive/MyDrive/Colab Notebooks/CaImAn/'

!pip install -e .

!pip install tifffile
!pip install ipyparallel
!pip install peakutils

#%cd '/content/CaImAn/'
%cd '/content/drive/MyDrive/Colab Notebooks/CaImAn/'
!python caimanmanager.py install --inplace

!export MKL_NUM_THREADS=1
!export OPENBLAS_NUM_THREADS=1

# This is for GPU accelaration
!pip install pycuda
!pip install scikit-cuda


!pip install pynwb # 要追加インストール


import caiman as cm 
"""
/caiman/base/movies.py 34 行目，"incremental_pcs" を "IncrementalPCA" に修正必要

"""


from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
import peakutils

# パラメータ設定

In [ ]:
# dataset dependent parameters
frate = 30                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between patches (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries
use_cuda = True         # Set to True in order to use GPU
only_init_patch = True
memory_fact = 0.8

mc_dict = {
    #'fnames': fnames,
    'fr': frate,
    'niter_rig': 1,
    'splits_rig': 20,  # for parallelization split the movies in  num_splits chuncks across time
    # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': None, # intervals at which patches are laid out for motion correction            
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'use_cuda' : use_cuda,
    'only_init_patch' : only_init_patch,
    'memory_fact': memory_fact
}

opts = params.CNMFParams(params_dict=mc_dict)